In [4]:
import logging

from src.bithumb.bithumb_api import BithumbApi
from src.common.google_sheet.cell_update import CellUpdate
from src.common.google_sheet.client import GoogleSheetClient
from src.config import BithumbConfig, GoogleSheetConfig, HantuConfig, UpbitConfig
from src.hantu import HantuDomesticAPI, HantuOverseasAPI
from src.hantu.model.domestic import AccountType
from src.upbit.upbit_api import UpbitAPI

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

upbit_api = UpbitAPI(UpbitConfig())  # type: ignore

hantu_domestic_api = HantuDomesticAPI(HantuConfig())  # type: ignore
hantu_overseas_api = HantuOverseasAPI(HantuConfig())  # type: ignore

v_hantu_domestic_api = HantuDomesticAPI(HantuConfig(), AccountType.VIRTUAL)  # type: ignore
v_hantu_overseas_api = HantuOverseasAPI(HantuConfig(), AccountType.VIRTUAL)  # type: ignore

auto_data_google_sheet_client = GoogleSheetClient(GoogleSheetConfig(), "auto_data")
kimp_data_google_sheet_client = GoogleSheetClient(GoogleSheetConfig(), "kimp_data")

bithumb_config = BithumbConfig()

2025-11-18 15:44:14,974 - urllib3.util.retry - DEBUG - Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
2025-11-18 15:44:15,016 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-11-18 15:44:15,555 - urllib3.connectionpool - DEBUG - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-11-18 15:44:15,566 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): sheets.googleapis.com:443
2025-11-18 15:44:16,490 - urllib3.connectionpool - DEBUG - https://sheets.googleapis.com:443 "GET /v4/spreadsheets/11xclEFkfaH01sAYjYRgnJS3-g8IPULy9Ypvkcip7N00?includeGridData=false HTTP/1.1" 200 None
2025-11-18 15:44:16,776 - urllib3.connectionpool - DEBUG - https://sheets.googleapis.com:443 "GET /v4/spreadsheets/11xclEFkfaH01sAYjYRgnJS3-g8IPULy9Ypvkcip7N00?includeGridData=false HTTP/1.1" 200 None
2025-11-18 15:44:16,826 - urllib3.util.retry - DEBUG - Converted retries value: 

In [6]:
import time
import uuid

import jwt
import requests

payload = {
    'access_key': bithumb_config.access_key,
    'nonce': str(uuid.uuid4()),
    'timestamp': round(time.time() * 1000)
}

headers = {
    'Authorization': f"Bearer {jwt.encode(payload, bithumb_config.secret_key)}"
}

apiUrl = 'https://api.bithumb.com'

try:
    response = requests.get(apiUrl + '/v1/accounts', headers=headers)
    # handle to success or fail
    print(response.status_code)
    print(response.json())
except Exception as err:
    print(err)

200
[{'currency': 'P', 'balance': '322', 'locked': '0', 'avg_buy_price': '0', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}, {'currency': 'KRW', 'balance': '1000001.534618', 'locked': '0', 'avg_buy_price': '0', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}, {'currency': 'USDT', 'balance': '3983.07775788', 'locked': '0', 'avg_buy_price': '1490', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}]


In [ ]:
from datetime import datetime, timedelta

from src.collector.data_fetcher import YfInterval, fetch_yfinance, fetch_finance_data_reader

usdt_df = upbit_api.get_candles("KRW-USDT", count=24 * 181)
krwusd_df = fetch_yfinance('KRW=X', interval=YfInterval.ONE_HOUR, start_date=datetime.now() - timedelta(days=181))


In [15]:
from src.collector.data_fetcher import fetch_finance_data_reader

fetch_finance_data_reader('GC=F')

2025-11-18 16:07:19,381 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query2.finance.yahoo.com:443
2025-11-18 16:07:19,877 - urllib3.connectionpool - DEBUG - https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/GC=F?period1=315500400&period2=1763449639&interval=1d&includeAdjustedClose=true HTTP/1.1" 200 None


,Open,High,Low,Close,Volume,Adj Close
2000-08-30,273.899994,273.899994,273.899994,273.899994,0.0,273.899994
2000-08-31,274.799988,278.299988,274.799988,278.299988,0.0,278.299988
2000-09-01,277.000000,277.000000,277.000000,277.000000,0.0,277.000000
2000-09-04,NaN,NaN,NaN,NaN,NaN,NaN
2000-09-05,275.799988,275.799988,275.799988,275.799988,2.0,275.799988
...,...,...,...,...,...,...
2025-11-12,4115.600098,4204.399902,4102.899902,4204.399902,396.0,4204.399902
2025-11-13,4228.700195,4228.700195,4163.700195,4186.899902,167.0,4186.899902
2025-11-14,4197.700195,4197.899902,4047.000000,4087.600098,483.0,4087.600098
2025-11-17,4070.000000,4099.500000,4019.399902,4068.300049,483.0,4068.300049


In [13]:
# kimp_data_google_sheet_client.batch_update_from_dataframe(usdt_df['close'], 2, 3)
# kimp_data_google_sheet_client.batch_update_from_dataframe(krwusd_df.tz_convert('Asia/Seoul').tz_localize(None)['Close'], 2, 1)

2025-11-18 15:52:48,772 - urllib3.connectionpool - DEBUG - Resetting dropped connection: sheets.googleapis.com
2025-11-18 15:52:50,708 - urllib3.connectionpool - DEBUG - https://sheets.googleapis.com:443 "POST /v4/spreadsheets/11xclEFkfaH01sAYjYRgnJS3-g8IPULy9Ypvkcip7N00/values:batchUpdate HTTP/1.1" 200 None
